## Write Data to CSV

In [1]:
import os
os.chdir('/Users/brianmcclanahan/git_repos/StrategyPortfolioTrading/portfolios/sml_equity')

In [2]:
import pandas as pd

df_norm = pd.read_parquet('ta_data.parquet')

In [4]:
df_norm.loc[:, 'date_formatted'] = df_norm.Date.dt.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
import numpy as np
cols = ['symbol', 'date_formatted', 'adj_close', 'Volume', 'adj_open', 'adj_high', 'adj_low', 'adj_close']
output_cols = ['symbol', 'Date Time', 'Adj Close', 'Volume', 'Open', 'High', 'Low', 'Close']
df_reduced = df_norm[cols].reset_index(drop=True)
df_reduced.columns = output_cols
df_reduced.loc[:, 'High'] = np.maximum(
    np.maximum(df_reduced.High, df_reduced.Open),
    df_reduced.Close
)
df_reduced.loc[:, 'Low'] = np.minimum(
    np.minimum(df_reduced.Low, df_reduced.Open),
    df_reduced.Close
)

In [6]:
csv_cols = ['Date Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
df_reduced.groupby('symbol').apply(lambda x: x.to_csv('all_cap/%s.csv' % x.symbol.iloc[0], index=False))

""


## Enter PyAlgo

In [7]:
from pyalgotrade.barfeed.csvfeed import GenericBarFeed
from pyalgotrade.bar import Frequency

barfeed = GenericBarFeed(Frequency.DAY)

In [8]:

import os

base_dirs = ['all_cap']

symbols = []
symbol_set = set([])

failed_symbols = []

for base_dir in base_dirs:
    for f in os.listdir(base_dir)[:]:
        symbol = f.split('.')[0]
        if symbol not in symbol_set:
            try:
                symbol_set.add(symbol)
                symbols.append(symbol)
                barfeed.addBarsFromCSV(symbol, '%s/%s' % (base_dir, f))
            except:
                failed_symbols.append(symbol)

In [9]:
len(failed_symbols)

1

In [10]:
from pyalgotrade import strategy
import numpy as np
import logging
import datetime as dt

class MRStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, entries, exits, equity_df,
                 score_sort=True,
                 pos_size=0.05, max_positions=2,
                 hold_period=100, stop=1.0,
                 cash_or_brk = 50000,
                 trail_stop = False,
                 weight_pos = True, drawdown_tresh=0.2,
                 drawdown_timeout=0,
                 loglevel=logging.ERROR):
        super(MRStrategy, self).__init__(feed, cash_or_brk=cash_or_brk)
        logger = self.getLogger()
        logger.setLevel(loglevel)
        self.max_positions = max_positions
        self.stop = stop
        self.hold_period = hold_period
        self.score_sort = score_sort
        self.trail_stop = trail_stop
        self.equity_df = equity_df
        self.pos_size = pos_size
        self.weight_pos = weight_pos
        self.drawdown_tresh = drawdown_tresh
        self.date_allowance = entries['date'].min()
        self.drawdown_timeout = drawdown_timeout
        self.__position = {}
        self.__position_hold = {}
        self.__position_strategy = {}
        self.__position_peak = {}
        self.__instruments = feed.getRegisteredInstruments()
        self.entries = entries.loc[entries.symbol.isin(self.__instruments)]
        self.exits = exits.loc[exits.symbol.isin(self.__instruments)]
        self.__tgt_pos_size = pos_size
        self.max_equity = 0
        self.getBroker().getFillStrategy().setVolumeLimit(None)
        for instrument in self.__instruments:
            self.__position[instrument] = None
            self.__position_hold[instrument] = 0
            self.__position_peak[instrument] = None
            
    def __clear_position(self, instrument):
        self.__position[instrument] = None
        self.__position_hold[instrument] = None
        self.__position_strategy[instrument] = None
        self.__position_peak[instrument] = None
        
    def onEnterOk(self, position):
        instrument = position.getInstrument()
        execInfo = position.getEntryOrder().getExecutionInfo()
        #self.info("BUY %s at $%.2f" % (instrument, execInfo.getPrice()))

    def onEnterCanceled(self, position):
        instrument = position.getInstrument()
        self.__clear_position(instrument)

    def onExitOk(self, position):
        instrument = position.getInstrument()
        execInfo = position.getExitOrder().getExecutionInfo()
        #self.info("SELL %s at $%.2f" % (instrument, execInfo.getPrice()))
        self.__clear_position(instrument)
        
    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        instrument = position.getInstrument()
        self.__position[instrument].exitMarket()
        
    def checkEntry(self, bars, instrument, pos_size, cash_available):
        bar = bars[instrument]
        close = bar.getClose()
        shares = int((self.getBroker().getEquity() * pos_size)/ close)
        if (shares * close) > cash_available:
            shares = 0
        else:
            cash_available -= shares * close
        return shares, close, cash_available
    
    def checkExit(self, bars, instrument, exit_df):
        exit = False
        bar = bars[instrument]
        close = bar.getClose()
        if close > self.__position_peak[instrument] and self.trail_stop:
            self.__position_peak[instrument] = close
        exec_info = self.__position[instrument].getEntryOrder().getExecutionInfo()
        if exec_info is not None:
            entry_price = exec_info.getPrice()
            exit = ((self.__position_peak[instrument] - close) / self.__position_peak[instrument]) > self.stop
            exit = exit or (self.__position_hold[instrument] < 1)
            exit = exit or (exit_df.shape[0] > 0)
        return exit
                
    def onBars(self, bars):
        current_dt = self.getCurrentDateTime()     
        entries = self.entries.loc[(self.entries['date'] == current_dt)]
        if current_dt == dt.datetime(2011, 1, 5):
            self.info(entries[['symbol', 'strategy_ind', 'pred_score']])
        if entries.shape[0] > 0:
            if self.score_sort == True:
                entries = entries.sort_values(['pred', 'volatility'], ascending=[False, False])
                #entries = entries.sort_values(['volatility'], ascending=[False])
            instruments_avail = entries.drop_duplicates(
                subset=['symbol'], keep='first'
            )[['symbol', 'strategy_ind', 'pred_score', 'volatility']].iloc[:self.max_positions]
            agg_score = instruments_avail.pred_score.sum()
            cash_available = self.getBroker().getCash()
            for ind in instruments_avail.index:
                sym = instruments_avail.loc[ind, 'symbol']
                strategy = instruments_avail.loc[ind, 'strategy_ind']
                pred_score = instruments_avail.loc[ind, 'pred_score']
                if self.__position[sym] is None:
                    weight = pred_score / agg_score if self.weight_pos else 1 / instruments_avail.shape[0]
                    weight *= self.pos_size * instruments_avail.shape[0]
                    shares, close, cash_available = self.checkEntry(bars, sym, weight, cash_available)
                    if shares > 0:
                        self.__position[sym] = self.enterLong(sym, shares, True)
                        self.__position_hold[sym] = self.hold_period
                        self.__position_strategy[sym] = strategy
                        self.__position_peak[sym] = close
                        self.entries.loc[ind, 'selected'] = 1
                
        instruments_avail = bars.keys()
        exits = self.exits.loc[self.exits['date'] == current_dt]
        cur_equity = self.getBroker().getEquity()
        max_equity = self.max_equity if self.max_equity > cur_equity else cur_equity
        drawdown = 1 - (cur_equity / max_equity)
        drawdown_hit = drawdown > self.drawdown_tresh
        max_equity = max_equity if not drawdown_hit else cur_equity
        if current_dt >= self.date_allowance:
            for instrument in instruments_avail:
                if self.__position[instrument] is not None:
                    strategy = self.__position_strategy[instrument]
                    exit_df = exits.loc[(exits.symbol == instrument) & (exits.strategy_ind == strategy)]
                    self.__position_hold[instrument] -= 1
                    exit = self.checkExit(bars, instrument, exit_df) or drawdown_hit
                    if exit and not self.__position[instrument].exitActive():
                        self.__position[instrument].exitMarket()
        if drawdown_hit:
            self.date_allowance = current_dt + dt.timedelta(days=self.drawdown_timeout)
        positions = self.getBroker().getPositions()
        self.max_equity = max_equity
        self.equity_df.loc[current_dt, 'drawdown'] = drawdown
        self.equity_df.loc[current_dt, 'drawdown_hit'] = drawdown_hit
        self.equity_df.loc[current_dt, 'max_equity'] = max_equity
        self.equity_df.loc[current_dt, 'equity'] = cur_equity
        self.equity_df.loc[current_dt, 'positions'] = len(self.getBroker().getPositions())
        self.equity_df.loc[current_dt, 'cash'] = self.getBroker().getCash()   

In [11]:
import pandas as pd
import datetime as dt

df = pd.read_parquet('backtesting/mean_reversion/small_mid_lrg_cap_signals.parquet')
date_range = pd.read_parquet('backtesting/mean_reversion/small_mid_lrg_cap_dt_rng.parquet')
df.loc[:, 'pred'] = 0
df.loc[df.pred_score >= 0.65, 'pred'] = 1
df = df.loc[df['date'] >= dt.datetime(2015, 1, 1)]
#equity_df = pd.DataFrame({'dates': date_range['date'].unique()}).sort_values('dates').set_index('dates')
equity_df = pd.DataFrame({'dates': date_range['date'].unique()}).sort_values('dates').set_index('dates')

equity_df.loc[:, 'equity'] = 0
entrances = df.loc[(df.actual_enter == 1)]
entrances.loc[:, 'selected'] = 0
exits = df.loc[df.exits == 1]
entrances['date'].unique().shape[0] / date_range['date'].unique().shape[0]

/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


0.2708071936428273

In [12]:
from pyalgotrade import plotter

mr_strategy = MRStrategy(barfeed, entrances, exits, equity_df, score_sort=True, stop=1.0)

In [13]:
from pyalgotrade.stratanalyzer import returns
from pyalgotrade.stratanalyzer import trades
from pyalgotrade.stratanalyzer import drawdown
from pyalgotrade.stratanalyzer import sharpe

returnsAnalyzer = returns.Returns(maxLen=365*40)
tradesAnalyzer = trades.Trades()
drawDownAnalyzer = drawdown.DrawDown()
sharpAnalyzer= sharpe.SharpeRatio()
mr_strategy.attachAnalyzer(returnsAnalyzer)
mr_strategy.attachAnalyzer(tradesAnalyzer)
mr_strategy.attachAnalyzer(drawDownAnalyzer)
mr_strategy.attachAnalyzer(sharpAnalyzer)
plt = plotter.StrategyPlotter(mr_strategy)

In [14]:
mr_strategy.run()

/Users/brianmcclanahan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
2015-02-03 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill ACOR order [33] for 59 share/s
2015-02-04 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill LJPC order [35] for 126 share/s
2015-05-28 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill REFR order [183] for 496 share/s
2015-05-29 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill CRBP order [185] for 893 share/s
2015-06-01 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill CRBP order [185] for 893 share/s
2015-06-08 00:00:00 broker.backtesting [DEBUG] Not enough cash to fill SHSP orde

In [ ]:
len(mr_strategy.getBroker().getPositions())

In [ ]:
import numpy as np
import pandas as pd
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go

def get_figure(sym, plt, rsi_strategy):
    instr = plt.getInstrumentSubplot(sym)
    instr_buy = instr.getSeries('Buy').getValues()
    instr_sell = instr.getSeries('Sell').getValues()
    bars = instr.getSeries(sym).getValues()
    buy_date, buy_price = zip(*instr_buy.items())
    sell_date, sell_price = zip(*instr_sell.items())
    buy_df = pd.DataFrame(
        {'buy_date': buy_date, 'buy_price': buy_price}
    ).sort_values('buy_date')
    sell_df = pd.DataFrame(
        {'sell_date': sell_date, 'sell_price': sell_price}
    ).sort_values('sell_date')
    bar_dict = {
        'open': [],
        'close': [],
        'high': [],
        'low': [],
        'bar_date': []
    }
    for dt in bars:
        bar = bars[dt]
        bar_dict['open'].append(bar.getOpen())
        bar_dict['close'].append(bar.getClose())
        bar_dict['high'].append(bar.getHigh())
        bar_dict['low'].append(bar.getLow())
        bar_dict['bar_date'].append(dt)
    bar_df = pd.DataFrame(bar_dict).sort_values('bar_date')
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.add_trace(
        go.Candlestick(open=bar_df.open, high=bar_df.high, low=bar_df.low,
                       close=bar_df.close, x=bar_df.bar_date, name='price'),
        row=1,col=1
    )
    fig.add_trace(
        go.Scatter(x=buy_df.buy_date, y=buy_df.buy_price, mode='markers', name='buy',
                   marker=dict(color='blue')),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=sell_df.sell_date, y=sell_df.sell_price, mode='markers', name='sell',
                   marker=dict(color='red')),
        row=1, col=1
    )
    fig.update_layout(xaxis_rangeslider_visible=False, title=sym)
    return fig

In [ ]:
fig = get_figure(symbols[3], plt, mr_strategy)
fig

In [ ]:
import matplotlib.pyplot as plt
rsi = np.array(rsi_strategy.rsi['AAOI'])
roc = np.array(rsi_strategy.roc['AAOI'])
x = np.arange(rsi.shape[0])
plt.plot(x, rsi)

In [ ]:
plt.plot(x, roc)

In [ ]:
import random
random.shuffle(symbols)
for sym in symbols[:30]:
    fig = get_figure(sym, plt, rsi_strategy) 
    plotly.offline.plot(fig, filename = 'pyalgo_charts/rsi/%s.html' % sym, auto_open=False)

In [15]:
import numpy as np
cum_returns = np.array(returnsAnalyzer.getCumulativeReturns())
returns = np.array(returnsAnalyzer.getReturns())

In [16]:
np.sum(returns), cum_returns[-1] #2.9, 9

(1.3010534551558837, 2.303683208845461)

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [ ]:
np.sum(returns), cum_returns[-1]

In [17]:
eq_df = mr_strategy.equity_df.sort_index()

In [ ]:
eq_df

In [18]:
eq_df.loc[:, 'peak'] = eq_df.equity.cummax()
eq_df.loc[:, 'drawdown'] = 1 - (eq_df.equity / eq_df.peak)

2020-12-25 19:41:58,850 numexpr.utils [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-12-25 19:41:58,850 numexpr.utils [INFO] NumExpr defaulting to 8 threads.


In [19]:
rsi = pd.read_parquet('ta_data.parquet')
plt_df = rsi.loc[rsi.symbol == 'A']

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#eq_df = mr_strategy.equity_df.sort_index()

def add_liquidation_lines(fig, eq_df, max_val, row_ind, col_ind):
    for row in eq_df.loc[eq_df.drawdown_hit.fillna(False)].index:
        fig.add_trace(
             go.Scattergl(
                    x=[row, row],
                    y=[0, max_val],
                    mode='lines',
                    line=dict(color='firebrick', dash='dash'),
                    showlegend=False
            ),
            row=row_ind, col=col_ind
        ) 


fig = make_subplots(rows=3, cols=1, shared_xaxes=True)#, specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scattergl(
        x=eq_df.index, y=eq_df.equity, name='equity curve'
    ),
    row=1, col=1
)

add_liquidation_lines(fig, eq_df, eq_df.equity.max(), 1, 1)

fig.add_trace(
    go.Scattergl(
        x=eq_df.index, y=eq_df.drawdown, name='drawdown'
    ),
    row=2, col=1
)

add_liquidation_lines(fig, eq_df, eq_df.drawdown.max(), 2, 1)

fig.add_trace(
    go.Scattergl(
        x=plt_df['Date'], y=plt_df.regime_ma, name='regime_ma'
    ),
    row=3, col=1
)

fig.add_trace(
    go.Scattergl(
        x=plt_df['Date'], y=plt_df.regime_close, name='regime_close'
    ),
    row=3, col=1
)
fig = fig.update_layout(width=1100, height=700)
fig

In [ ]:

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(
    go.Scattergl(
        x=plt_df['date'], y=plt_df.regime_ma, name='regime_ma'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scattergl(
        x=plt_df['date'], y=plt_df.regime_close, name='regime_close'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scattergl(
        x=eq_df.index, y=eq_df.equity, name='equity curve'
    ),
    row=2, col=1
)
fig = fig.update_layout(width=1100, height=700)
fig

In [ ]:
import plotly.graph_objects as go

eq_df = mr_strategy.equity_df.sort_index()

fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eq_df.index, y=eq_df.positions, name='positions'
    ),
    secondary_y=True
)
fig

In [ ]:
mr_strategy.equity_df.sort_index().to_parquet('equity_curves/stop_15_20_pos.parquet')

In [ ]:
#mr_strategy.equity_df.sort_index().to_parquet('equity_curves/stop.parquet')

In [ ]:
eq_df2 = pd.read_parquet('equity_curves/no_stop.parquet')

In [ ]:
eq_df3 = pd.read_parquet('equity_curves/stop.parquet')

In [ ]:
eq_df3.loc[equity_df.equity != 50000].iloc[-1]

In [ ]:
eq_df2.loc[equity_df.equity != 50000].iloc[-1]

In [ ]:
eq_df.loc[equity_df.equity != 50000].iloc[-1]

In [ ]:
import plotly.graph_objects as go

eq_df = mr_strategy.equity_df.sort_index()

fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eq_df.index, y=eq_df.equity, name='equity curve no stop'
    )
)
fig.add_trace(
    go.Scattergl(
        x=eq_df2.index, y=eq_df2.equity, name='equity curve no stop larger pos'
    )
)
fig.add_trace(
    go.Scattergl(
        x=eq_df3.index, y=eq_df3.equity, name='equity curve'
    )
)
fig

In [ ]:
import plotly.graph_objects as go

eq_df = mr_strategy.equity_df.sort_index()

fig = go.Figure()

fig.add_trace(
    go.Scattergl(
        x=eq_df.index, y=eq_df.equity, name='equity curve'
    )
)
fig

In [ ]:
def print_stats():
    trade_count = tradesAnalyzer.getCount()
    wins_cnt = tradesAnalyzer.getProfitableCount()
    loss_cnt = tradesAnalyzer.getUnprofitableCount()
    profits = tradesAnalyzer.getProfits()
    losses = tradesAnalyzer.getLosses()
    norm_profits = tradesAnalyzer.getPositiveReturns()
    norm_losses = tradesAnalyzer.getNegativeReturns()
    all_returns = tradesAnalyzer.getAllReturns()
    sharp_ratio = sharpAnalyzer.getSharpeRatio(.02)
    max_drawdown = drawDownAnalyzer.getMaxDrawDown()

    measures = {
        'win_rate': (wins_cnt) / (wins_cnt + loss_cnt),
        'win count': wins_cnt,
        'loss count': loss_cnt,
        'mean_profit': profits.mean(),
        'mean_loss': losses.mean(),
        'mean_norm_profit': norm_profits.mean(),
        'mean_norm_loss': norm_losses.mean(),
        'mean_return': all_returns.mean(),
        'sharp_ratio': sharp_ratio,
        'max_drawdown': max_drawdown,
        'cum_returns': cum_returns[-1],
        'avg_cum_return': np.power(cum_returns[-1], 1 / 10) - 1,
        'total_cash': mr_strategy.getBroker().getEquity()
    }
    return pd.Series(measures).to_frame().transpose() 

In [ ]:
print_stats()

In [ ]:
print_stats()

In [ ]:
print_stats()

In [ ]:
print_stats()

In [ ]:
print_stats()

In [ ]:
mr_strategy.entries.loc[mr_strategy.entries.selected == 1].to_parquet('actual_entries.parquet')

In [ ]:
entrancies.to_parquet('actual_entries')

In [ ]:
import matplotlib.pyplot as plt

x  = np.arange(returns.shape[0])
plt.plot(x, returns)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

x  = np.arange(returns.shape[0])
plt.plot(x, cum_returns)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

x  = np.arange(returns.shape[0])
plt.plot(x, returns)
plt.show()

In [ ]:
import plotly.graph_objs as go



In [ ]:
import matplotlib.pyplot as plt
x  = np.arange(returns.shape[0])
plt.plot(x, cum_returns)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

x  = np.arange(returns.shape[0])
plt.plot(x, returns)
plt.show()

In [ ]:
x  = np.arange(returns.shape[0])
plt.plot(x, cum_returns)
plt.show()

In [ ]:
cum_returns[-1]

In [ ]:
cum_returns[-1]

In [ ]:
cum_returns[-1]

In [ ]:
print("Final portfolio value: $%.2f" % rsi_strategy.getBroker().getEquity())